During this exercise, it was important to constuct a function with minimum functionality so it could be easily used with other functions.

In [1]:
import numpy as np
from myfunctions import *

Available functions: downlad, login 
Available object: headers(dict with my user-agent info)


In [72]:
import networkx as nx
from bs4 import BeautifulSoup
import copy
import matplotlib.pyplot as plt
from pyvis.network import Network

first section

In [3]:
url = 'https://docs.scipy.org/doc/numpy/reference/index.html'
html = download('get', url)
dom = BeautifulSoup(html.text, 'html')

In [4]:
def makeDict(base, dom, level):
    foo = dict()
    foo['name'] = dom.find('a').text
    foo['url'] = base + dom.find('a').get_attribute_list('href')[0]
    tempList = list()
    for i in dom.findAll('li', {'class':'toctree-l2'}):
        temp = dict()
        temp['name'] = i.text
        temp['url'] = i.find('a').get_attribute_list('href')[0]
        tempList.append(temp)
    foo[level] = tempList
    return foo

In [7]:
###########################################################################################
# 마중물
d = {"name": "Numpy Reference", 
     "url": "https://docs.scipy.org/doc/numpy/reference/index.html"}
base = "https://docs.scipy.org/doc/numpy/reference/"
def mine(d, level):
    html = download('get', d['url'])
    dom = BeautifulSoup(html.text, 'html')
    body = dom.find('div', {'id':'spc-section-body'})
    foo = dict()
    foo['name'] = d['name']
    
    if body.find('div', {'class':'toctree-wrapper compound'}):
        reference = dom.findAll('li', {"class":"toctree-l1"})
        templist= list()
        
        for i in reference:
            templist.append(makeDict(base, i, level))
        body.find('div', {'class':'toctree-wrapper compound'}).decompose() # does not return
        foo['content'] = body.text 
        foo['child'] = templist
    else:
        foo['content'] = body.text
    return foo

In [22]:
a = mine(d, 'firstchildren')

In [25]:
for x, i in zip(a['child'], range(len(a['child']))):
    print(i)
    b = mine(x, 'secondchildren')
    print(b.keys())
    a['child'][i]['content'] = b['content']
    try:
        a['child'][i]['firstchildren'] = b['child']
    except:
        continue

0
dict_keys(['content', 'name', 'child'])
1
dict_keys(['content', 'name'])
2
dict_keys(['content', 'name'])
3
dict_keys(['content', 'name', 'child'])
4
dict_keys(['content', 'name'])
5
dict_keys(['content', 'name'])
6
dict_keys(['content', 'name', 'child'])
7
dict_keys(['content', 'name', 'child'])
8
dict_keys(['content', 'name', 'child'])


In [14]:
b = copy.deepcopy(a)

In [15]:
a = copy.deepcopy(b)

In [57]:
for first, x in zip(a['child'], range(len(a['child']))):
    print(first['name'], '____')
    
    for second, y in zip(first['firstchildren'], range(len(first['firstchildren']))):
#         print(second['name'])
        if 'secondchildren' in a['child'][x]['firstchildren'][y].keys():
            h = download('get', a['child'][x]['firstchildren'][y]['url'])
            dom = BeautifulSoup(h.text, 'html')
            body = dom.find('div', {'id':'spc-section-body'})
            
            a['child'][x]['firstchildren'][y]['content'] = body.text
        else:
            continue

Array objects ____
Constants ____
Universal functions (ufunc) ____
Routines ____
Packaging (numpy.distutils) ____
NumPy Distutils - Users Guide ____
NumPy C-API ____
NumPy internals ____
NumPy and SWIG ____


In [101]:
n = []
e = []
DG = Network()
DG.add_node(a['name'], color = '#063c7a')
for i in a['child']:
#     print(i['name'], "------")
    DG.add_node(i['name'], color = '#9ec0e8')
    DG.add_edge(a['name'], i['name'])
    for x in i['firstchildren']:
#         print(x['name'])
        DG.add_node(x['name'], color= '#bae6ce')
        DG.add_edge(i['name'], x['name'])
        
        if 'secondchildren' in x.keys():
            for y in x['secondchildren']:
                DG.add_node(y['name'], color = '#d4e6a5')
                DG.add_edge(x['name'], y['name'])

In [103]:
DG.show('numpy_doc_networkx.html')

--END--